# マイナビ2021スクレイピング
（分析しか使わないのでご容赦ください！）

IT・通信
* https://job.mynavi.jp/22/pc/search/query.html?IC:40,41,42/LICM:1/func=PCTopQuickSearch

流れ
* 一覧ページのリンクを取得する
* リストへ格納する
* 「次の100社へ」をクリック
* 一覧のページを取得する　
* リストへ格納する

で一通りの企業際ページのリンクを取得する

* 取得したリンクの企業のページを確認
    * 会社名
    * 本社所在地
    * メールアドレス

を取得し、格納するをリンク分繰り返す

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from time import sleep

options = Options()
options.add_argument('--headless') # ブラウザを立ち上げない
browser = webdriver.Chrome('/Users/yoshimi/Documents/Python/ml_sample/chromedriver',options=options)

pageURL = 'https://job.mynavi.jp/21/pc/search/query.html?IC:40,41,42/LICM:1/func=PCTopQuickSearch'
browser.get(pageURL)
sleep(3)

In [2]:
# リンク一覧を取得する

# リンク一覧を格納するリストを準備
elem_urls=[]

# while で　find_element_by_link_text　が　breakするまでループ
while True:
    elems = browser.find_elements_by_css_selector(".boxSearchresultEach h3 a")

    for elem in elems:
        elem_urls.append(elem.get_attribute("href"))

    # 次へをクリックしページ遷移する
    try:
        next_button = browser.find_element_by_link_text('次の100社')
        next_button.click()
        sleep(3)
    except Exception:
        #browser.quit()
        break
        
print('会社ページ数：', len(elem_urls))

会社ページ数： 2727


In [3]:
# 列を指定し、空のデータフレーム作成し、代入
cols=['会社名','本社','E-mail']
df = pd.DataFrame(index=[], columns=cols)

#options = Options()
#options.add_argument('--headless') # ブラウザを立ち上げない
#browser = webdriver.Chrome('/Users/yoshimi/Documents/Python/ml_sample/chromedriver',options=options)

for i in elem_urls:
    browser.get(i)

    # 会社名取得
    comp_tit = browser.find_element_by_class_name("heading1-inner")
    comp_tit = comp_tit.find_element_by_tag_name("h1").text
    # 本社住所
    comp_add = browser.find_element_by_id("corpDescDtoListDescText50").text
    # E-mail
    try:
        comp_email = browser.find_element_by_id("corpDescDtoListDescText130").text
        #comp_email = comp_email.split('：')[1]
    except NoSuchElementException:
        comp_email = ''

    df=df.append({'会社名':comp_tit,'本社':comp_add,'E-mail':comp_email}, ignore_index=True)
    

print(len(df))

2727


In [21]:
df.tail(10)

,会社名,本社,E-mail
2717,(株)デジマース,東京都品川区東五反田3-20-14 高輪パークタワー,koutaki@digimerce.jp
2718,(株)CLIS,東京都品川区南大井6-26-1,saiyo@clis.co.jp
2719,(株)シンク,福岡県福岡市博多区博多駅前2-19-24 大博センタービル5階,
2720,(株)オプテージ,大阪市中央区城見2丁目1番5号 オプテージビル,recruit@optage.co.jp
2721,ナガセ情報開発(株),東京都中央区日本橋本町1-2-8 長瀬産業本町ビル,nid@nagase.co.jp
2722,ジャパンシステム(株)【東証JASDAQ上場】,東京都渋谷区代々木1-22-1 代々木1丁目ビル3F,js_new@japan-systems.co.jp
2723,日野コンピューターシステム(株)【日野自動車グループ】,日野市日野台1-16-12,kanri-h@hcs.hino.co.jp
2724,(株)イーエフユーアドウィス,大阪府大阪市西区靭本町1-4-8 本町KAZビル 9F,recruit@efu-addwis.co.jp
2725,(株)プライムソフトサービス,名古屋市中区栄2丁目8番12号 伏見KSビル2階,primesofts@primesoftservice.com
2726,朝日ソフトウェア開発(株),神奈川県川崎市川崎区東田町8番地 パレール三井ビル15階,
